In [15]:
## imports

import numpy as np
import pandas as pd

import statsmodels.formula.api as sm

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
root = os.path.expanduser('~')

company_index='5'
company_name='Amazon'
img_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/img/companies/'+f'{company_index}_{company_name}'
img_flag = False

if not os.path.exists(img_path):
    os.makedirs(img_path)

In [16]:
def merge_data(company_list=[5]):
    df_ = pd.DataFrame()
    df_company_list = pd.read_csv( root + '/Desktop/workspace/indeed/Job-Satisfaction/data/scraper_data/review_site.csv')
    for company_index in company_list:
        company_name = df_company_list.iloc[company_index - 1]['Company_Name']
        data_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}/{company_name}.csv'
        df_ = pd.concat([df_, pd.read_csv(data_path)]).reset_index(drop=True)
    return df_

# df = merge_data(company_list=[5, 6])
# df = merge_data(company_list=[1, 2, 3, 4, 5])
df = merge_data(company_list=range(1, 51))





In [17]:
# filter only needed columns

df_rating = df[['Rating_Number', 'Work_Life', 'Benefits', 'Job_Advancement', 'Management', 'Culture']]
df_rating.dropna(inplace=True)
df_rating.head()
df_rating.shape

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(682176, 6)

In [18]:
result = sm.ols(formula="Rating_Number ~ Work_Life + Benefits + Job_Advancement + Management + Culture", 
                data=df_rating).fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          Rating_Number   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                 8.471e+04
Date:                Mon, 22 Jul 2019   Prob (F-statistic):               0.00
Time:                        13:19:24   Log-Likelihood:            -9.1846e+05
No. Observations:              682176   AIC:                         1.837e+06
Df Residuals:                  682170   BIC:                         1.837e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.1681      0.003    7

In [19]:
display(result.params)

Intercept          2.168063
Work_Life          0.122071
Benefits          -0.032949
Job_Advancement    0.082079
Management         0.174596
Culture            0.149845
dtype: float64

In [22]:
result.tvalues

Intercept          779.614870
Work_Life           94.717061
Benefits           -28.263582
Job_Advancement     60.590226
Management         125.064762
Culture            103.575477
dtype: float64

In [28]:
result.pvalues[2]

1.2250906537837227e-175

In [25]:
result.rsquared

0.3830451821324288

In [11]:
ind_vars = df_rating.columns.tolist()
ind_vars.remove('Rating_Number')
ind_vars

['Work_Life', 'Benefits', 'Job_Advancement', 'Management', 'Culture']

In [30]:
df_results = pd.DataFrame(columns=['Dimension', 'Coefficient', 'T-Value', 'P-Value', 'R-Squared'])
index = 0
for var in ind_vars:
    result = sm.ols(formula=f"Rating_Number ~ {var}", 
                data=df_rating).fit()
    df_results.loc[index] = [var, result.params[1], result.tvalues[1], result.pvalues[1], result.rsquared]
    index += 1
df_results.head()

,Dimension,Coefficient,T-Value,P-Value,R-Squared
0,Work_Life,0.417333,546.474742,0.0,0.304478
1,Benefits,0.345820,432.643163,0.0,0.215309
2,Job_Advancement,0.403813,531.940419,0.0,0.293183
3,Management,0.429898,592.395786,0.0,0.339687
4,Culture,0.428032,584.986002,0.0,0.334063
